# 5.7.1 신경망 학습의 전체 그림

신경망 학습은 아래의 순서로 반복된다.

1. 미니배치 - 훈련 데이터 중 일부를 가져옴
2. 기울기 산출 - 가중치 매개변수의 기울기를 구함
3. 매개변수 갱신 - 가충치 매개변수를 기울기 방향으로 조금 갱신

위 3단계를 계속 반복한다.

# 5.7.2 오차 역전파법을 이용한 신경망 구현하기

* OrderedDict 용법 알기
* Affine 계층 용법 알기
* 역전파 때 미분값의 최초값은 1이다. (진짜 1이었다니...)

In [5]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # ★★★★ 1 ★★★★ 
        # 순서가 있는 딕셔너리
        self.layers = OrderedDict() 

        # ★★★★ 2 ★★★★
        # Affine은 x * W + b 해주는 역할
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) 
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1 # ★★★★ 3 ★★★★ 
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


In [13]:
two = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in reversed(two.layers.values()):
    print(i)

print('----')

for j in two.layers.values():
    print(j)


----


# 5.7.3 오차 역전파법으로 기울기 검증하기

In [19]:
import sys, os
import numpy as np
from dataset.mnist import load_mnist
import time

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

# 실제로 학습할 때는 아래 두 줄 중 하나만 써야 함.

stime = time.time()
grad_numerical = network.numerical_gradient(x_batch, t_batch)
print(time.time() - stime)

stime = time.time()
grad_backprop = network.gradient(x_batch, t_batch)
print(time.time() - stime)

# 오차 평균
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

0.0
W1:0.0006751023017552469
b1:0.0035592502616701538
W2:0.006811786361545174
b2:0.00030747053899727597


가중치의 오차가 매우 작다. 그러니 여기서의 포인트는 수치미분으로 구한 기울기함수와 우리가 오차 역전파로 계산한 기울기 값이 거의 동일하다는 것을 보여준다. 그럼 오차역전파를 굳이 쓰는 이유는? 우선 빠르다. time.time을 넣어서 비교해보면 어마어마하게 차이가 난다.

* 수치미분은 계산이 느리다.
* 우리가 이번 장에서 구현한 오차역전파는 행렬로 계산을 한꺼번에 한다!

# 5.7.4 학습 구현하기

단순히 수치미분을 오차역전파로 바꿔준 것이다. 각자 구현해보면 좋다!

In [23]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    # grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식 ★★★★★
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.11875 0.1232
0.9020833333333333 0.908


KeyboardInterrupt: 